# Luokittelu (GradientBoostingClassifier)

JAMK examples #11: 11_ml_revenue_boosting

Ladataan käytettävät paketit.

In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

Luetaan käytettävä aineisto:

In [2]:
csvUrl = "https://raw.githubusercontent.com/RaimoHaikari/paikkatieto/master/misc/sales_data.csv"
df = pd.read_csv(csvUrl)

df.shape

(1000, 6)

Seuraavaksi:

- imputoidaan numeeristen muuttujie NA -arvot muuttujien keskiarvoilla (EI OLE)
- lasketaan päivittäinen liikevaihto
- muunnetaan aineisto aikasarjaksi

In [7]:
# Handle missing values
# df = df.dropna()
df.fillna(df.mean(numeric_only=True), inplace=True)

# Calculate the daily revenue
df['revenue'] = df['sales'] * df['price']

Luokitellaan aineisto liikevaihdon mukaan kolmeen luokkaan:

- **Low** myynti on alle 1000 
- **Medium** myynti on välillä 1000 ... 5000
- **High** myynti on yli 5000

In [8]:
# Define revenue classes based on limits
low_limit = 1000
high_limit = 5000
df['revenue_class'] = pd.cut(
    df['revenue'], 
    bins=[-float('inf'), low_limit, high_limit, float('inf')],
    labels=['Low', 'Medium', 'High']
)

Erotetaan syöte- ja vastemuuttujan omiin muuttujiinsa.

In [9]:
# Prepare the data
X = df[['price', 'category']]  # Features
y = df['revenue_class']  # Target variable

Dummy-koodataa luokkamuuttujat.

In [10]:
# Perform one-hot encoding on the 'category' column
X = pd.get_dummies(X)

Jaetaan aineisto opetus- ja testijoukkoihin.

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Alustetaan luokittelua varten Gradient Boosting -luokittelija, jonka jälkeen sovitetaan malli.

In [12]:
# Initialize the Gradient Boosting classifier model
model = GradientBoostingClassifier()

model.fit(X_train, y_train)

GradientBoostingClassifier()

Testataan mallia testijoukolla.

In [13]:
# Make predictions on the test set
y_pred = model.predict(X_test)

sklearn sisältää [classification_report-funktion](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html), joka tulostaa luokittelusta perusrapsan:

- **precision** kertoo onnistuneiden ennusteiden osuuden (ENNUSTE = TODELLINEN)
- **recall** kertoo kuinka hyvin todellisien arvojen näkökulmasta ennusteet osuivat (TODELLINEN = ENNUSTE)
- **f1-score** on precision ja recall arvojen yhdistelmä. (Näyttäisi olevan puolessa välissä)
- **support** luokkien todellisten ilmentymien lkm
- **accuracy** kuinka suuri osuus ennusteista osui oikeaan, kun huomioidaan kaikki luokat

- **macro avg** kaikkien ennusteluokkien onnistumisprosenttien keskiarvo, niin ennusteen, todellisten kuin näiden yhteenvedon näkökulmasta.
- **weighted avg** painottaa ennusteiden arvoja luokkafrekvenssien mukaisesti, [mitä ei yleensä haluta, koska harvinaisemmat luokat yleensä vaikeampia tunnistaa](https://datascience.stackexchange.com/questions/65839/macro-average-and-weighted-average-meaning-in-classification-report).


In [14]:
# Evaluate the model
classification_report = classification_report(y_test, y_pred)
print('Classification Report:\n', classification_report)

Classification Report:
               precision    recall  f1-score   support

        High       0.50      0.52      0.51        33
         Low       0.67      0.48      0.56        60
      Medium       0.65      0.75      0.70       107

    accuracy                           0.63       200
   macro avg       0.61      0.58      0.59       200
weighted avg       0.63      0.63      0.62       200

